### Libs & Data

In [9]:
import pandas as pd
import numpy as np
import os
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import string
import random
import unicodedata
from string import punctuation
from string import digits
from nltk.stem import WordNetLemmatizer
import joblib

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import PCA
# from sklearn.decomposition import TruncatedSVD
# from sklearn.manifold import TSNE
# from sklearn.cluster import KMeans
# from sklearn.cluster import DBSCAN
# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt

import torch
# from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

import spacy
from spacy.matcher import Matcher

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kwsst\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kwsst\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
# Load large spacy model 
nlp = spacy.load('en_core_web_lg')

In [11]:
df2015 = pd.read_csv("data/df2015.csv")
df2016 = pd.read_csv("data/df2016.csv")
df2017 = pd.read_csv("data/df2017.csv")
df2017.head()

,title,author,time,description,body,section,summarized_body,label
0,A snapshot of multiculturalism in South Korea,Lee Sun-young,2018-01-01 17:07:00,With birthrates persistently low and the senio...,With birthrates persistently low and the senio...,Social affairs,"As of 2016, more than 2 million foreign nation...",0
1,[Weekender] Korea’s dynamic 2017,Choi He-suk,2018-01-01 13:22:00,From North Korea’s nuclear weapons program nea...,From North Korea’s nuclear weapons program nea...,Social affairs,The corruption scandal that broke out in 2016 ...,1
2,People's Party members support Ahn's push for ...,Yonhap,2017-12-31 16:18:00,The leader of the center-left People's Party g...,The leader of the center-left People's Party g...,Politics,"Ahn Cheol-soo, leader of the center-left Peopl...",2
3,[Newsmaker] Panamanian vessel probed over susp...,Yonhap,2017-12-31 14:55:00,PYEONGTAEK -- South Korea has seized and insp...,PYEONGTAEK -- South Korea has seized and insp...,North Korea,"The 5,100-ton KOTI is being held in the wester...",3
4,Hong Kong ship crew questioned in S. Korea for...,AFP,2017-12-30 15:44:00,The crew of a Hong Kong-registered ship have b...,The crew of a Hong Kong-registered ship have b...,North Korea,The crew of a Hong Kong-registered ship have b...,4


In [12]:
top20sum = pd.read_csv("data/top20sum.csv")
top20sum.head()

,2015,label2015,freq2015,2016,label2016,freq2016,2017,label2017,freq2017
0,South Korea reported no additional cases of Mi...,2957,6,"Radio Pyongyang, the North's state-run radio s...",16,5,Temperatures across the country plummeted abou...,67,11
1,"In October, some 36,900 babies were born, up 1...",132,5,Some 200 chickens were found dead on Monday mo...,518,4,A South Korean research team says it has uncov...,617,7
2,"Cho Hyun-ah, former vice president of Korean A...",6018,4,Voter turnout in South Korea’s parliamentary e...,5095,4,South Korean scientists have developed an adhe...,4735,5
3,Seoul-Tokyo ties have plunged to lowest levels...,5457,4,Walkway along Deoksu Palace in downtown Seoul ...,705,3,"About 30,300 babies were born in May, down 11....",3708,5
4,South Korea's top financial regulator said Fri...,5039,4,Pigs at two swine farms in Nonsan in the centr...,5782,3,"The search for the missing South Korean ship, ...",6654,4


### NER

In [13]:
# Returns merged sentences given df and label
def merge_sents(df=df2015, cluster='label2015'):
    sents = []
    for label in top20sum[cluster]:
        merged_sent = ""
        for sent in df[df.label == label]['summarized_body']:
            merged_sent += sent
        sents.append(merged_sent)
    return sents
sents2015 = merge_sents(df2015, 'label2015')
sents2016 = merge_sents(df2016, 'label2016')
sents2017 = merge_sents(df2017, 'label2017')

In [14]:
top20sum['sents2015'] = sents2015
top20sum['sents2016'] = sents2016
top20sum['sents2017'] = sents2017
top20sum
top20sum.to_csv("data/top20sum_sents.csv", index=False)

In [15]:
# Extract entities from sentences
from collections import Counter
def extract_entity(sents=sents2015, clusters=top20sum['label2015']):
    for i in range(len(sents)):
        doc = nlp(sents[i])
        ent_list = dict()
        for ent in doc.ents:
            if not ent.label_ in ['MONEY','TIME','CARDINAL','QUANTITY','DATE','ORDINAL','PERCENT']:
                if ent.label_ in ent_list:
                    ent_list[ent.label_].append(ent.text)
                else:
                    ent_list[ent.label_] = [ent.text]
        print("Cluster id {}".format(clusters[i]))
        for k in ent_list:
            v = ent_list[k]
            ent_counter = Counter(v)
            top10item = ent_counter.most_common(10)
            print("\t{}: {}".format(k, top10item))

In [16]:
extract_entity(sents=sents2015, clusters=top20sum['label2015'])

Cluster id 2957
	GPE: [('South Korea', 4)]
	LOC: [('Middle East', 5)]
	PERSON: [('MERS', 4)]
	ORG: [('the Ministry of Health and Welfare', 1)]
Cluster id 132
	GPE: [('South Korea', 5)]
Cluster id 6018
	PERSON: [('Cho Hyun-ah', 3)]
	ORG: [('Cho', 2), ('Korean Air', 1)]
	GPE: [('Cho', 1), ('New York', 1), ('Seoul', 1)]
Cluster id 5457
	GPE: [('Japan', 5), ('Tokyo', 5), ('Seoul', 4), ('South Korea', 2), ('Beijing', 1), ('Korea', 1)]
	NORP: [('Korean', 3), ('South Korean', 1)]
	EVENT: [('World War II', 3)]
Cluster id 5039
	GPE: [("South Korea's", 3), ('Korea', 1)]
	NORP: [('South Koreans', 1)]
	ORG: [('the Bank of Korea', 1)]
Cluster id 3617
	NORP: [('Koreans', 1), ('South Korean', 1)]
	EVENT: [('the Yuan Dynasty', 1)]
	ORG: [('the Humane Society International', 2), ('the Change for Animals Foundation', 2)]
	GPE: [('South Korea', 2), ('S. Korea', 1), ('U.S.', 1)]
Cluster id 5864
	GPE: [('Seoul', 3), ('Gyeonggi Province', 2), ('Incheon', 1), ('Gangwon Province', 1)]
	ORG: [('Citizens', 1), 

In [18]:
extract_entity(sents=sents2016, clusters=top20sum['label2016'])

Cluster id 16
	ORG: [('Radio Pyongyang', 3)]
	LOC: [('North', 1)]
	GPE: [('North Korea', 3), ('Korea', 1), ('Seoul', 1), ("North Korea's", 1), ('South Korea', 1), ('Pyongyang', 1)]
	EVENT: [('the Cold War', 1)]
Cluster id 518
	GPE: [('Yangju', 2), ('Eumseong', 1), ('South Korea', 1), ('Gyeonggi Province', 1), ('Haenam', 1), ('South Jeolla Province', 1)]
	LOC: [('South Chungcheong Province', 1), ('the west coastline', 1)]
	ORG: [('The Ministry of Agriculture,', 1), ('Food and Rural Affairs', 1), ('Ministry of Food, Agriculture and Rural Affairs', 1)]
	PERSON: [('Gimje', 1)]
Cluster id 5095
	NORP: [('South Korean', 1)]
	GPE: [('South Korea’s', 3)]
Cluster id 705
	GPE: [('Seoul', 2), ('Deoksugung', 1), ('Korea', 1), ('U.K.', 1), ('Seoul City', 1)]
	ORG: [('the British Embassy', 2), ('the Seoul Metropolitan Government', 1), ('Seoul Metropolitan Government', 1)]
	FAC: [('Stone Wall Walkway', 1), ('Deoksu Palace', 1), ('Deoksugung Palace', 1)]
Cluster id 5782
	LOC: [('Nonsan', 2), ('South Ch

In [19]:
extract_entity(sents=sents2017, clusters=top20sum['label2017'])

Cluster id 67
	GPE: [('Gangwon Province', 4), ('Seoul', 4), ('Gyeonggi Province', 3), ('North Gyeongsang Province', 2), ('Jeju Island', 2), ('Jeolla Province', 1), ('Korea', 1), ('Chungcheong Province', 1), ('North Jeolla Province', 1)]
	LOC: [('North Chungcheong Province', 1)]
	ORG: [('The Korea Meteorological Administration', 2), ('Cheongju', 1), ('Daejeon', 1), ('C, Gwangju', 1), ('KMA', 1), ('C.A', 1)]
Cluster id 617
	NORP: [('Korean', 2), ('South Korean', 1), ('Japanese', 1)]
	LOC: [('the Korean Peninsula', 1), ('Korean Peninsula', 1)]
	ORG: [('Twitter', 4), ('Homepage', 2), ('Mount Baekdu', 1), ('Seoul Zoo', 1), ('AmurLeopard', 1), ('The National Institute of Biological Resources', 1)]
	GPE: [('China', 1), ('North Korea', 1), ('South Korea', 1)]
	PERSON: [('Gregory Pence', 2), ('Mount Jiri', 1)]
	LAW: [('@SBComics', 2)]
Cluster id 4735
	NORP: [('South Korean', 5)]
	PERSON: [('Bang Chang-hyun', 1), ('Han Chang-soo', 1), ('Lee Hyun-jae', 1)]
	ORG: [('Sungkyunkwan University', 1), (

### Experiment

In [24]:
df2016[df2016.label == 16]['title'].tolist()

['N. Korea resumes encrypted numbers broadcast with same content as 2 weeks ago',
 'N. Korea resumes encrypted numbers broadcast after 4-day break',
 'NK resumes encrypted numbers broadcast after 8-day break',
 'NK resumes encrypted numbers broadcast after 9-day break',
 'NK resumes encrypted numbers broadcast after 2-week hiatus']

In [25]:
df2016[df2016.label == 16].index.tolist()

[17, 230, 909, 1163, 1382]

In [44]:
df2016[df2016.label == 16]['body'].tolist()[0]

'North Korea\'s state radio station resumed broadcasting mysterious numbers Friday in what could be some kind of coded message to its agents operating in South Korea.Radio Pyongyang, the North\'s state-run radio station, started broadcasting messages at 1:15 a.m. (Seoul time), calling out a series of pages and numbers, such as No. 69 on Page 894, before repeating them one more time.The radio announcer said, "(I\'m giving) review works in math lessons of the remote education university for No. 27 expedition agents." The content was the same as those transmitted in the early hours of Dec. 16.Since June 24, 20 of such encrypted numbers broadcasts have been discovered, with the latest one broadcast Sunday. (Yonhap)Broadcasts of mysterious numbers are considered a kind of book cipher that was often used by North Korea to give missions to spies operating in South Korea during the Cold War era. Spies could decode numbers to get orders by using a reference book, although many intelligence offi

In [34]:
def extract(sent, clusters=top20sum['label2015']):
    doc = nlp(sent)
    ent_list = dict()
    for ent in doc.ents:
        if not ent.label_ in ['MONEY','TIME','CARDINAL','QUANTITY','DATE','ORDINAL','PERCENT']:
            if ent.label_ in ent_list:
                ent_list[ent.label_].append(ent.text)
            else:
                ent_list[ent.label_] = [ent.text]
    print("Cluster id {}".format(16))
    for k in ent_list:
        v = ent_list[k]
        ent_counter = Counter(v)
        top10item = ent_counter.most_common(10)
        print("\t{}: {}".format(k, top10item))

for i in range(5):
    print(extract(sent = df2016[df2016.label == 16]['body'].tolist()[i],
                  clusters = top20sum['label2016']))

Cluster id 16
	GPE: [('South Korea', 3), ("North Korea's", 1), ('Seoul', 1), ('North Korea', 1), ('Pyongyang', 1), ('Koreas', 1)]
	ORG: [('Radio Pyongyang', 1)]
	LOC: [('North', 1)]
	EVENT: [('the Cold War', 1)]
None
Cluster id 16
	GPE: [('South Korea', 3), ('North Korea', 2), ("North Korea's", 1), ('Korea', 1), ('Pyongyang', 1), ('Koreas', 1)]
	ORG: [('Radio Pyongyang', 1), ('Musudan', 1), ('Yonhap', 1)]
	EVENT: [('the Cold War', 1)]
None
Cluster id 16
	GPE: [('South Korea', 3), ('North Korea', 3), ("North Korea's", 1), ('Seoul', 1), ('Pyongyang', 1), ('Koreas', 1)]
	ORG: [('Radio Pyongyang', 1), ('Musudan', 1), ('Yonhap', 1)]
	EVENT: [('the Cold War', 1)]
None
Cluster id 16
	GPE: [('South Korea', 3), ('North Korea', 3), ("North Korea's", 1), ('Seoul', 1), ('Pyongyang', 1), ('Koreas', 1)]
	ORG: [('Radio Pyongyang', 1), ('Musudan', 1), ('Yonhap', 1)]
	EVENT: [('the Cold War', 1)]
None
Cluster id 16
	GPE: [('Seoul', 3), ('North Korea', 3), ('South Korea', 2), ("North Korea's", 1), ('Pyo